In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # Loads variables from .env

api_key = os.getenv("TOGETHER_API_KEY")


In [3]:
!pip install pdfplumber
!pip install together

  Using cached pdfplumber-0.11.6-py3-none-any.whl.metadata (42 kB)
  Using cached pdfminer_six-20250327-py3-none-any.whl.metadata (4.1 kB)
  Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl.metadata (48 kB)
  Using cached cryptography-44.0.2-cp39-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached pdfplumber-0.11.6-py3-none-any.whl (60 kB)
Using cached pdfminer_six-20250327-py3-none-any.whl (5.6 MB)
Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl (3.0 MB)
Using cached cryptography-44.0.2-cp39-abi3-win_amd64.whl (3.2 MB)
Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl (181 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)



[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached together-1.5.5-py3-none-any.whl.metadata (14 kB)
  Using cached aiohttp-3.11.18-cp312-cp312-win_amd64.whl.metadata (8.0 kB)
  Using cached eval_type_backport-0.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached pillow-11.2.1-cp312-cp312-win_amd64.whl.metadata (9.1 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached typer-0.15.3-py3-none-any.whl.metadata (15 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.6.0-cp312-cp312-win_amd64.whl.metadata (16 kB)
  Using cached multidict-6.4.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached propcache-0.3.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached yarl-1.20.0-cp312-cp312-win_amd64.whl.metadata (74 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires torch, which is not installed.
easyocr 1.7.2 requires torchvision>=0.5, which is not installed.
streamlit 1.34.0 requires pillow<11,>=7.1.0, but you have pillow 11.2.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pdfplumber

def extract_pdf_text(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text()
    return full_text

# Test the extraction
pdf_path = "/content/SathishKumar_Data Scientist.pdf"
pdf_text = extract_pdf_text(pdf_path)
print(pdf_text)

ModuleNotFoundError: No module named 'pdfplumber'

In [ ]:
import requests
import json
import pdfplumber


# Define prompt
prompt = f"""
You are an expert resume parser.

Task:
Parse the following resume content and extract structured fields. Return the result ONLY in the following strict JSON format:

Instructions:
- Do not guess. If a field is not explicitly present in the text, set its value to the JSON null value (without quotes).
- Read the ENTIRE content carefully, especially the profile/summary and projects section, to extract all relevant skills.
- Add skills from projects or summary sections **only if they are not already included** in the skills list.
- LinkedIn: Only extract a LinkedIn URL if it is a valid personal LinkedIn profile (e.g., starts with "https://linkedin.com/in/username"). If it's just the word "LinkedIn", a logo, or a general link (e.g., "linkedin.com"), set the field to null.
- Work Experience vs. Projects:
  - Treat as **work experience** only if BOTH a valid company name and a clear duration (e.g., "Jan 2020 – Mar 2022") are present.
  - If any of the following are **missing** — company name, duration, or job title — classify it as a **project**.
  - Terms like "Project", "Capstone", "Intern Project", "Practical Experience", or job simulation **should be treated as projects**, NOT experience.
- Extract project title and description from such entries and move them to the "projects" section.
- Normalize all dates to "MMM YYYY" format where possible (e.g., "Jan 2020").
- Include a confidence score (0.0–1.0) for each field in the `confidence_scores` section.
- Return all results ONLY in this strict JSON format:

{{
  "name": "",
  "email": "",
  "phone": "",
  "linkedin": "",
  "skills": [],
  "education": [
    {{
      "degree": "",
      "institution": "",
      "year": ""
    }}
  ],
  "experience": [
    {{
      "company": "",
      "title": "",
      "duration": "",
      "description": ""
    }}
  ],
  "certifications": [],
  "projects": [
    {{
      "title": "",
      "description": ""
    }}
  ],
  "confidence_scores": {{
    "name": 0.0,
    "email": 0.0,
    "phone": 0.0,
    "linkedin": 0.0,
    "skills": 0.0,
    "education": 0.0,
    "experience": 0.0,
    "certifications": 0.0,
    "projects": 0.0
  }}
}}

Text to extract from:
\"\"\"
{pdf_text}
\"\"\"
"""

# Call Together API
response = requests.post(
    "https://api.together.xyz/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    },
    json={
        "model": "mistralai/Mistral-7B-Instruct-v0.2",
        "messages": [
            {"role": "system", "content": "You are an expert in reading resumes."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "top_p": 0.9,
        "max_tokens": 2048,
    }
)

# Parse the output
output = response.json()

# Print the full response
print(json.dumps(output, indent=2))

# Print just the result
print("\n\nResult:\n")
print(output['choices'][0]['message']['content'])


{
  "id": "ns1SFj8-3NKUce-937d4b71f9c51d64",
  "object": "chat.completion",
  "created": 1745913930,
  "model": "mistralai/Mistral-7B-Instruct-v0.2",
  "prompt": [],
  "choices": [
    {
      "finish_reason": "stop",
      "seed": 15242759706418150000,
      "logprobs": null,
      "index": 0,
      "message": {
        "role": "assistant",
        "content": " {\n  \"name\": \"SATHISH KUMAR\",\n  \"email\": \"2310sathishkumarsk@gmail.com\",\n  \"phone\": \"+91 8825743005\",\n  \"linkedin\": \"null\",\n  \"skills\": [\n    \"Python\",\n    \"NumPy\",\n    \"Pandas\",\n    \"SQL\",\n    \"MySQL\",\n    \"Machine Learning\",\n    \"PyTorch\",\n    \"Scikit-learn\",\n    \"Power BI\",\n    \"Data Analysis\",\n    \"Natural Language Processing(NLP)\",\n    \"Data Preprocessing\",\n    \"Feature Engineering\"\n  ],\n  \"education\": [\n    {\n      \"degree\": \"Bachelor of Business Administration\",\n      \"institution\": \"Vidyasagar College of Arts and Science Udumalpet, India\",\n    

In [ ]:
raw_output = output['choices'][0]['message']['content']

In [ ]:
import json

# Convert string to a Python dictionary
parsed_output = json.loads(raw_output)

# Post-process the output to replace string "null" with actual None (null in JSON)
def clean_json_output(parsed_output):
    for key, value in parsed_output.items():
        if isinstance(value, dict):
            parsed_output[key] = clean_json_output(value)  # Recurse if dictionary
        elif isinstance(value, list):
            for i in range(len(value)):
                if isinstance(value[i], dict):
                    value[i] = clean_json_output(value[i])  # Recurse if dictionary inside list
                elif value[i] == "null":
                    value[i] = None  # Replace "null" string with actual null (None in Python)
        elif value == "null":
            parsed_output[key] = None  # Replace "null" string with actual null (None in Python)
    return parsed_output


In [ ]:
# Clean the parsed output
cleaned_output = clean_json_output(parsed_output)

# Print cleaned output
print(json.dumps(cleaned_output, indent=2))

{
  "name": "SATHISH KUMAR",
  "email": "2310sathishkumarsk@gmail.com",
  "phone": "+91 8825743005",
  "linkedin": null,
  "skills": [
    "Python",
    "NumPy",
    "Pandas",
    "SQL",
    "MySQL",
    "Machine Learning",
    "PyTorch",
    "Scikit-learn",
    "Power BI",
    "Data Analysis",
    "Natural Language Processing(NLP)",
    "Data Preprocessing",
    "Feature Engineering"
  ],
  "education": [
    {
      "degree": "Bachelor of Business Administration",
      "institution": "Vidyasagar College of Arts and Science Udumalpet, India",
      "year": "2019 \u2013 2022"
    },
    {
      "degree": "Advanced Programming Professional & Master Data Science",
      "institution": "IIT-M GUVI Chennai, India",
      "year": "2023 \u2013 2024"
    }
  ],
  "experience": [
    {
      "company": null,
      "title": "Data Science Specialist Intern",
      "duration": null,
      "description": "Eager to contribute to GUVI\u2019s mission of accessible vernacular tech learning."
    }
  

## focus Image based pdf

In [1]:
!pip install pytesseract pdf2image

In [2]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.7 [186 kB]
Fetched 186 kB in 0s (891 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.7_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.7) ...
Setting up poppler-utils (22.02.0-2ubuntu0.7) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
from pdf2image import convert_from_path
import pytesseract

# Path to your PDF file
pdf_path = '/content/r01.pdf'

# Convert PDF to images
images = convert_from_path(pdf_path)

# OCR the images and extract text
extracted_text = ""
for img in images:
    text = pytesseract.image_to_string(img)
    extracted_text += text

print(extracted_text)  # Display extracted text


SATHISH KUMAR

Data Scientist

& 2310sathishkumarsk@gmail.com &, +91 8825743005 @ Tiruppur, India ff LinkedIn €) Github

PROFILE

Motivated and detail-oriented Data Science enthusiast with a solid foundation in Python, SQL, Machine Learning, and
Deep Learning. Trained under GUVI's IIT-M incubated program and completed real-world projects focusing on data-
driven decision-making. Eager to contribute to GUVI’s mission of accessible vernacular tech learning as a Data Science
Specialist Intern.

SKILLS

Python | NumPy | Pandas | SQL | MySQL | Machine Learning | PyTorch | Scikit-learn | Power BI |
Data Analysis | Natural Language Processing (NLP) | Data Preprocessing | Feature Engineering

PRACTICAL EXPERIENCE

Telecom Churn Prediction and Analysis &

Key Skills: Python, Machine Learning, Power BI, EDA, Data Visualization

e Developed a predictive model to identify telecom customers at risk of churn, allowing early intervention.

+ Performed EDA on 6,418 customer records (27% churn rate) to

In [17]:
import re

def preprocess_ocr(text):
    text = re.sub(r"[^\x00-\x7F]+", "", text)  # Remove non-ASCII
    text = re.sub(r"[&*@#€¢•¶]+", "", text)     # Remove common OCR junk
    #text = re.sub(r"\s{2,}", " ", text)         # Replace multiple spaces with one
    return text.strip()

cleaned_text = preprocess_ocr(extracted_text)


In [18]:
print(cleaned_text)

SATHISH KUMAR

Data Scientist

 2310sathishkumarskgmail.com , +91 8825743005  Tiruppur, India ff LinkedIn ) Github

PROFILE

Motivated and detail-oriented Data Science enthusiast with a solid foundation in Python, SQL, Machine Learning, and
Deep Learning. Trained under GUVI's IIT-M incubated program and completed real-world projects focusing on data-
driven decision-making. Eager to contribute to GUVIs mission of accessible vernacular tech learning as a Data Science
Specialist Intern.

SKILLS

Python | NumPy | Pandas | SQL | MySQL | Machine Learning | PyTorch | Scikit-learn | Power BI |
Data Analysis | Natural Language Processing (NLP) | Data Preprocessing | Feature Engineering

PRACTICAL EXPERIENCE

Telecom Churn Prediction and Analysis 

Key Skills: Python, Machine Learning, Power BI, EDA, Data Visualization

e Developed a predictive model to identify telecom customers at risk of churn, allowing early intervention.

+ Performed EDA on 6,418 customer records (27% churn rate) to uncove

In [ ]:
import requests
import json
import pdfplumber


# Define prompt
prompt = f"""
You are an expert resume parser.

Task:
Parse the following resume content and extract structured fields. Return the result ONLY in the following strict JSON format:

Instructions:
- Do not guess. If a field is not explicitly present in the text, set its value to the JSON null value (without quotes).
- Read the ENTIRE content carefully, especially the profile/summary and projects section, to extract all relevant skills.
- Add skills from projects or summary sections **only if they are not already included** in the skills list.
- LinkedIn: Only extract a LinkedIn URL if it is a valid personal LinkedIn profile (e.g., starts with "https://linkedin.com/in/username"). If it's just the word "LinkedIn", a logo, or a general link (e.g., "linkedin.com"), set the field to null.
- Work Experience vs. Projects:
  - Treat as **work experience** only if BOTH a valid company name and a clear duration (e.g., "Jan 2020 – Mar 2022") are present.
  - If any of the following are **missing** — company name, duration, or job title — classify it as a **project**.
  - Terms like "Project", "Capstone", "Intern Project", "Practical Experience", or job simulation **should be treated as projects**, NOT experience.
- Extract project title and description from such entries and move them to the "projects" section.
- Normalize all dates to "MMM YYYY" format where possible (e.g., "Jan 2020").
- Include a confidence score (0.0–1.0) for each field in the `confidence_scores` section.
- Return all results ONLY in this strict JSON format:

{{
  "name": "",
  "email": "",
  "phone": "",
  "linkedin": "",
  "skills": [],
  "education": [
    {{
      "degree": "",
      "institution": "",
      "year": ""
    }}
  ],
  "experience": [
    {{
      "company": "",
      "title": "",
      "duration": "",
      "description": ""
    }}
  ],
  "certifications": [],
  "projects": [
    {{
      "title": "",
      "description": ""
    }}
  ],
  "confidence_scores": {{
    "name": 0.0,
    "email": 0.0,
    "phone": 0.0,
    "linkedin": 0.0,
    "skills": 0.0,
    "education": 0.0,
    "experience": 0.0,
    "certifications": 0.0,
    "projects": 0.0
  }}
}}

Text to extract from:
\"\"\"
{cleaned_text}
\"\"\"
"""

# Call Together API
response = requests.post(
    "https://api.together.xyz/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    },
    json={
        "model": "mistralai/Mistral-7B-Instruct-v0.2",
        "messages": [
            {"role": "system", "content": "You are an expert in reading resumes."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "top_p": 0.9,
        "max_tokens": 2048,
    }
)

# Parse the output
output = response.json()

# Print the full response
print(json.dumps(output, indent=2))

# Print just the result
print("\n\nResult:\n")
print(output['choices'][0]['message']['content'])


{
  "id": "ns2pmGo-3NKUce-937ecaf0d9d1e829",
  "object": "chat.completion",
  "created": 1745929638,
  "model": "mistralai/Mistral-7B-Instruct-v0.2",
  "prompt": [],
  "choices": [
    {
      "finish_reason": "stop",
      "seed": 3843311838032930000,
      "logprobs": null,
      "index": 0,
      "message": {
        "role": "assistant",
        "content": " {\n  \"name\": \"SATHISH KUMAR\",\n  \"email\": \"2310sathishkumarskgmail.com\",\n  \"phone\": \"+91 8825743005\",\n  \"linkedin\": null,\n  \"skills\": [\"Python\", \"NumPy\", \"Pandas\", \"SQL\", \"MySQL\", \"Machine Learning\", \"PyTorch\", \"Scikit-learn\", \"Power BI\", \"Data Analysis\", \"Natural Language Processing (NLP)\", \"Data Preprocessing\", \"Feature Engineering\"],\n  \"education\": [\n    {\n      \"degree\": \"Advanced Programming Professional Master Data Science\",\n      \"institution\": \"IIT-M GUVI Chennai, India\",\n      \"year\": \"2023 - 2024\"\n    },\n    {\n      \"degree\": \"Bachelor of Business Ad

In [24]:
raw_output = output['choices'][0]['message']['content']

In [25]:
import json

# Convert string to a Python dictionary
parsed_output = json.loads(raw_output)

# Post-process the output to replace string "null" with actual None (null in JSON)
def clean_json_output(parsed_output):
    for key, value in parsed_output.items():
        if isinstance(value, dict):
            parsed_output[key] = clean_json_output(value)  # Recurse if dictionary
        elif isinstance(value, list):
            for i in range(len(value)):
                if isinstance(value[i], dict):
                    value[i] = clean_json_output(value[i])  # Recurse if dictionary inside list
                elif value[i] == "null":
                    value[i] = None  # Replace "null" string with actual null (None in Python)
        elif value == "null":
            parsed_output[key] = None  # Replace "null" string with actual null (None in Python)
    return parsed_output


In [26]:
# Clean the parsed output
cleaned_output = clean_json_output(parsed_output)

# Print cleaned output
print(json.dumps(cleaned_output, indent=2))

{
  "name": "SATHISH KUMAR",
  "email": "2310sathishkumarskgmail.com",
  "phone": "+91 8825743005",
  "linkedin": null,
  "skills": [
    "Python",
    "NumPy",
    "Pandas",
    "SQL",
    "MySQL",
    "Machine Learning",
    "PyTorch",
    "Scikit-learn",
    "Power BI",
    "Data Analysis",
    "Natural Language Processing (NLP)",
    "Data Preprocessing",
    "Feature Engineering"
  ],
  "education": [
    {
      "degree": "Advanced Programming Professional Master Data Science",
      "institution": "IIT-M GUVI Chennai, India",
      "year": "2023 - 2024"
    },
    {
      "degree": "Bachelor of Business Administration",
      "institution": "Vidyasagar College of Arts and Science Udumalpet, India",
      "year": "2019  2022"
    }
  ],
  "experience": [
    {
      "company": null,
      "title": "Data Science Specialist Intern",
      "duration": null,
      "description": "Eager to contribute to GUVIs mission of accessible vernacular tech learning."
    }
  ],
  "certification

In [28]:
import re

def fix_email(email):
    if email and "@" not in email:
        # Try inserting '@' before 'gmail' or 'yahoo' etc.
        match = re.search(r"(\w+)(gmail|yahoo|outlook|protonmail|hotmail)\.com", email)
        if match:
            corrected = match.group(1) + "@" + match.group(2) + ".com"
            return corrected
    return email

cleaned_output["email"] = fix_email(cleaned_output["email"])


In [29]:
cleaned_output

{'name': 'SATHISH KUMAR',
 'email': '2310sathishkumarsk@gmail.com',
 'phone': '+91 8825743005',
 'linkedin': None,
 'skills': ['Python',
  'NumPy',
  'Pandas',
  'SQL',
  'MySQL',
  'Machine Learning',
  'PyTorch',
  'Scikit-learn',
  'Power BI',
  'Data Analysis',
  'Natural Language Processing (NLP)',
  'Data Preprocessing',
  'Feature Engineering'],
 'education': [{'degree': 'Advanced Programming Professional Master Data Science',
   'institution': 'IIT-M GUVI Chennai, India',
   'year': '2023 - 2024'},
  {'degree': 'Bachelor of Business Administration',
   'institution': 'Vidyasagar College of Arts and Science Udumalpet, India',
   'year': '2019  2022'}],
 'experience': [{'company': None,
   'title': 'Data Science Specialist Intern',
   'duration': None,
   'description': 'Eager to contribute to GUVIs mission of accessible vernacular tech learning.'}],
 'certifications': ['Python for Data Science - IBM',
  'Data Analytics Essentials - CISCO',
  'Microsoft Power BI - Guvi'],
 'proje